# Project 2C: Ames Housing Data & Kaggle Challenge

### Overall Contents:
- Background
- Data Cleaning
- Exploratory Data Analysis
- [Modeling](#4.-Visualize-the-Data) **(In this notebook)**
- [Conclusions and Recommendations](#5.-Conclusions-and-Recommendations) **(In this notebook)**

### Data Dictionary
The dataset used for this analysis are as followed:-

* [`train`]: Ames Housing Train Data - labeled as housing_data

Data source: [Ames Housing Data](https://www.kaggle.com/c/dsi-us-11-project-2-regression-challenge/data) obtained from database [Ames Iowa Assessor's Office](http://www.cityofames.org/assessor/)

|Feature|Type|Dataset|Description|
|:------|:---:|:---:|:---|
|overall_qual|integer|Housing|The overall material and finish of the house| 
|exter_qual|integer|Housing|Evaluates the quality of the material on the exterior| 
|bsmt_qual|integer|Housing|Evaluates the height of the basement| 
|total_bsmt_sf|integer|Housing|Total square feet of basement area|
|heating_qc|integer|Housing|Heating quality and condition| 
|gr_liv_area|integer|Housing|Above grade (ground) living area square feet|
|full_bath|integer|Housing|Full bathrooms above grade (ground)|  
|kitchen_qual|integer|Housing|Kitchen quality| 
|fireplace_qu|integer|Housing|Fireplace quality| 
|garage_finish|integer|Housing|Interior finish of the garage| 
|garage_area|integer|Housing|Size of garage in square feet| 
|log_saleprice|float|Housing|Logarithm of the sale price of the house| 
|age|integer|Housing|Age of the house|
|bsmt_qual_total|integer|Housing|Combine effect of total basement area and its quality| 
|arage_area_finish|integer|Housing|Combine effect of garage area and garage finish|
|fire_heat_qc|integer|Housing|Combine effect of Fireplace quality and heating quality| 
|totrms_abvgrd|integer|Housing|Total rooms above ground|
|ms_zoning|integer|Housing|The general zoning classification of the sale|
||||- FV  Floating Village Residential|        
||||- I   Industrial|   
||||- RH  Residential High Density|        
||||- RL  Residential Low Density|      
||||- RM  Residential Medium Density| 


## 4. Modeling

### 4.1 Libraries Import

In [1]:
# Imports:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression, Lasso, LassoCV, Ridge, RidgeCV
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler

%config InlineBackend.figure_format = 'retina'
%matplotlib inline 

### 4.2 Data Import

In [2]:
# Import of Ames Housing Train Data from csv
train_housing_data = pd.read_csv('../datasets/potential_features_id.csv')
test_housing_data = pd.read_csv('../datasets/test_potential_features_id.csv')

In [3]:
train_housing_data.head()

,id,overall_qual,exter_qual,bsmt_qual,total_bsmt_sf,heating_qc,gr_liv_area,full_bath,kitchen_qual,fireplace_qu,...,log_saleprice,age,bsmt_qual_total,garage_area_finish,fire_heat_qc,ms_zoning_FV,ms_zoning_I,ms_zoning_RH,ms_zoning_RL,ms_zoning_RM
0,109,6,3,3,725,4,1479,2,3,0,...,11.779129,34,2175,950,0,0,0,0,1,0
1,544,7,3,4,913,4,2122,2,3,3,...,12.301383,14,3652,1118,12,0,0,0,1,0
2,153,5,2,3,1057,2,1057,1,3,0,...,11.599103,57,3171,246,0,0,0,0,1,0
3,318,5,2,4,384,3,1444,2,2,0,...,12.066811,4,1536,1200,0,0,0,0,1,0
4,255,6,2,2,676,2,1445,2,2,0,...,11.838626,110,1352,484,0,0,0,0,1,0


In [4]:
test_housing_data.head()

,id,overall_qual,exter_qual,bsmt_qual,total_bsmt_sf,heating_qc,gr_liv_area,full_bath,kitchen_qual,fireplace_qu,...,garage_area,age,bsmt_qual_total,garage_area_finish,fire_heat_qc,ms_zoning_FV,ms_zoning_I,ms_zoning_RH,ms_zoning_RL,ms_zoning_RM
0,2658,6,2,2,1020,3,1928,2,1,0,...,440,100,2040,440,0,0,0,0,0,1
1,2718,5,2,4,1967,2,1967,2,2,0,...,580,33,7868,1740,0,0,0,0,1,0
2,2414,7,3,4,654,4,1496,2,3,4,...,426,4,2616,852,16,0,0,0,1,0
3,1989,5,3,3,968,2,968,1,2,0,...,480,87,2904,480,0,0,0,0,0,1
4,625,6,2,4,1394,3,1394,1,2,4,...,514,47,5576,1028,12,0,0,0,1,0


### 4.3 Model the data 

We will be performing a train/test split on our data set to have a training set and a holdout set. In total, we will use three models (LinearRegression, Lasso Regression, Ridge Regression) to evaluate and choose the best model prior to fitting the dataset to predict the y-values from the validation set. After the evaluation, we will then proceed to perform a prediction of y-values of our test set for submission to Kaggle.

#### 4.3.1 Split the data into train/test data

In [6]:
train_housing_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2006 entries, 0 to 2005
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  2006 non-null   int64  
 1   overall_qual        2006 non-null   int64  
 2   exter_qual          2006 non-null   int64  
 3   bsmt_qual           2006 non-null   int64  
 4   total_bsmt_sf       2006 non-null   int64  
 5   heating_qc          2006 non-null   int64  
 6   gr_liv_area         2006 non-null   int64  
 7   full_bath           2006 non-null   int64  
 8   kitchen_qual        2006 non-null   int64  
 9   fireplace_qu        2006 non-null   int64  
 10  garage_finish       2006 non-null   int64  
 11  garage_area         2006 non-null   int64  
 12  log_saleprice       2006 non-null   float64
 13  age                 2006 non-null   int64  
 14  bsmt_qual_total     2006 non-null   int64  
 15  garage_area_finish  2006 non-null   int64  
 16  fire_h

In [8]:
test_housing_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 878 entries, 0 to 877
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   id                  878 non-null    int64
 1   overall_qual        878 non-null    int64
 2   exter_qual          878 non-null    int64
 3   bsmt_qual           878 non-null    int64
 4   total_bsmt_sf       878 non-null    int64
 5   heating_qc          878 non-null    int64
 6   gr_liv_area         878 non-null    int64
 7   full_bath           878 non-null    int64
 8   kitchen_qual        878 non-null    int64
 9   fireplace_qu        878 non-null    int64
 10  garage_finish       878 non-null    int64
 11  garage_area         878 non-null    int64
 12  age                 878 non-null    int64
 13  bsmt_qual_total     878 non-null    int64
 14  garage_area_finish  878 non-null    int64
 15  fire_heat_qc        878 non-null    int64
 16  ms_zoning_FV        878 non-null    int64
 1

In [9]:
# Creating an X and Y variable for train set
y_data = train_housing_data['log_saleprice']
x_data = train_housing_data.drop(columns = ['id',"log_saleprice"])

In [13]:
#Verify Dimensions
print('X train: ', x_data.shape)
print('y train: ', y_data.shape)

X train:  (2006, 21)
y train:  (2006,)


In [12]:
#Verify Dimensions for test_housing_data
print('X test : ', test_housing_data.shape)

X test :  (878, 21)


To prevent confusion, X_val and y_val will be used instead of X_test and y_test. This is to differentiate from the test data set naming.

In [14]:
# Perform the train-test-split on the train set with a train_size of 0.7
X_train, X_val, y_train, y_val = train_test_split(x_data,y_data, train_size = 0.7, random_state = 7, shuffle = True)

In [15]:
#Verify Dimensions
print('X train: ', X_train.shape)
print('y train: ', y_train.shape)
print('X val: ', X_val.shape)
print('y val: ', y_val.shape)
print('x test: ',test_housing_data.shape)

X train:  (1404, 21)
y train:  (1404,)
X val:  (602, 21)
y val:  (602,)
x test:  (878, 21)


### 4.2 Standardscaler

As we will be performing lasso and ridge model, we will need to standardscale the features. 

We first fit a standardscaler to the train data set and transform both x_train and X_test set.

In [16]:
# Initiate a blank StandardScaler
sc = StandardScaler()

In [17]:
# Fit and transform into X_train prior fitting into X_test
z_train = sc.fit_transform(X_train)
z_val = sc.transform(X_val)
z_test = sc.transform(test_housing_data)

### 4.3 Instantiate the model

Prior to evaluating the models, we will need to create instances of these models

In [18]:
# Instantiate the Linear Regression model
lr = LinearRegression()

In [19]:
# Instantiate the Lasso Regression model
lasso = LassoCV (n_alphas = 200)

In [20]:
# Instantiate the Ridge Regression model
ridge = RidgeCV(alphas= np.logspace(0, 5, 200), cv = 5, scoring = 'r2')

### 4.4 Cross validation

We perform the evaluation of the these models using the cross_val_score

In [21]:
# Perform a cross_val_score on the train_set for Linear Regression
lr_scores = cross_val_score(lr, z_train, y_train, cv = 5)
print(f"The linear regression cross_val_score is {lr_scores.mean()}")

The linear regression cross_val_score is 0.867139939654631


In [22]:
# Perform a cross_val_score on the train_set for Lasso Regression
lasso_scores = cross_val_score(lasso, z_train, y_train, cv = 5)
print(f"The lasso regression cross_val_score is {lasso_scores.mean()}")

The lasso regression cross_val_score is 0.8665843752472681


In [23]:
# Perform a cross_val_score on the train_set for Ridge Regression
ridge_scores = cross_val_score(ridge, z_train, y_train, cv = 5)
print(f"The ridge regression cross_val_score is {ridge_scores.mean()}")

The ridge regression cross_val_score is 0.8679540009558497


**Analysis: Ridge Regression performing the best with an R score of about 0.866.**

The cross_val_score across these models are approximately the same with a score of 0.866 with Ridge Regression performing better than the rest with 0.868. The Ridge Regressions functions by penalizing the size of the weights by introducing a regularization parameter. This model is useful for our analysis in which we have many predictors that may have a high degree of Multicollinearity between each other [[1]](https://towardsdatascience.com/from-linear-regression-to-ridge-regression-the-lasso-and-the-elastic-net-4eaecaf5f7e6) However, using ridge regression requires standard scaling and select an optimal alpha (hyperparameter) for its peak performance. 

Despite that, this regression is preferable over Lasso Regression as the cross validation score for this model is the highest out of the three and it helps to shrinks our regression coefficients closer to zero by making our model simpler.

### 4.5 Model Fitting and Evaluation

With our chosen Ridge model, we will proceed to fit our data to the model and evaluate our model further.

In [24]:
# Fit the standard scaled x_train and y_train to ridge
ridge.fit(z_train, y_train)

RidgeCV(alphas=array([1.00000000e+00, 1.05956018e+00, 1.12266777e+00, 1.18953407e+00,
       1.26038293e+00, 1.33545156e+00, 1.41499130e+00, 1.49926843e+00,
       1.58856513e+00, 1.68318035e+00, 1.78343088e+00, 1.88965234e+00,
       2.00220037e+00, 2.12145178e+00, 2.24780583e+00, 2.38168555e+00,
       2.52353917e+00, 2.67384162e+00, 2.83309610e+00, 3.00183581e+00,
       3.18062569e+00, 3.37006433e+0...
       2.64308149e+04, 2.80050389e+04, 2.96730241e+04, 3.14403547e+04,
       3.33129479e+04, 3.52970730e+04, 3.73993730e+04, 3.96268864e+04,
       4.19870708e+04, 4.44878283e+04, 4.71375313e+04, 4.99450512e+04,
       5.29197874e+04, 5.60716994e+04, 5.94113398e+04, 6.29498899e+04,
       6.66991966e+04, 7.06718127e+04, 7.48810386e+04, 7.93409667e+04,
       8.40665289e+04, 8.90735464e+04, 9.43787828e+04, 1.00000000e+05]),
        cv=5, scoring='r2')

In [25]:
# The r2 score of our train set using ridge
ridge.score(z_train, y_train)

0.8750581652273024

In [26]:
# The r2 score of our val score using ridge
ridge.score(z_val,y_val)

0.8826168433401163

In [27]:
# Make predictions for test_housing_data for Kaggle submissions
test_housing_data_pred = ridge.predict(test_housing_data)

In [36]:
test_housing_data_pred

array([ 385.73073314,  707.84577835,  355.20501802,  321.48233331,
        501.69968719,  240.50018365,  307.08483709,  458.82631934,
        402.96391231,  394.58948505,  456.75716123,  331.95181207,
        442.47466405,  659.54955442,  390.63081181,  178.43437852,
        364.07592148,  338.47854575,  469.89070667,  382.89746063,
        328.20740734,  289.85010229,  580.24853909,  328.2290692 ,
        443.96842041,  284.44992158,  344.53504844,  309.29365012,
        305.32148028,  164.97756998,  295.64863871,  305.68300158,
        553.47849917,  435.38597969,  540.23078543,  329.64881329,
        422.70637583,  151.03511986,  152.80141803,  506.70082235,
        303.23814791,  456.01452549,  382.99451196,  355.16120461,
        545.49384481,  263.27756867,  444.20673852,  295.23729606,
        284.8276111 ,  314.42092058,  331.25249434,  322.11020152,
        592.37057782,  334.3447601 ,  226.52980857,  320.00140126,
        411.38372261,  372.84511014,  373.33360345,  474.38610

In [29]:
kaggle_test = pd.DataFrame(test_housing_data["id"])

In [35]:
kaggle_test["saleprice"] = np.exp(test_housing_data_pred)

<ipython-input-35-27d71b2e4506>:1: RuntimeWarning: overflow encountered in exp
  kaggle_test["saleprice"] = np.exp(test_housing_data_pred)


In [34]:
kaggle_test.to_csv('submit_kaggle_chinxia.csv', index=False)

**Analysis: Train score, test score and cross val score are similar with about 87-88%, which indicates the model prediction is reliable.**

Both our train and test score is about 0.87, which indicates that 87% of the variability in sale price can be explained by the x predictors in our model. While it is observed that the train score is higher than the test score. This might indicate that there is a non distributed proportion of sample split in the test and train set resulting in the test set having an overly-representated sample. Thus, the sample in the test sets fair better in being predicted.

On the other hand, it is observed that the train and test score is higher than the cross val score. This observation is possible as the cross val score is a better estimator as it performs k-fold cross validation by performing multiple train/test splits across different splits of the data, which would be a more representative score in comparison with the score of a single train/test split. Nevertheless, the score of the single train/test split of about 0.87 with the cross val score of 0.867 indicates the score is similar and thus, the model prediction is reliable.

#### 4.5.1 To check out and interpret our model coefficient and intercept

In [ ]:
# Plot of coefficients of X predictors
pd.Series(((np.exp(ridge.coef_)-1)*100), index = x_data.columns).plot.barh(figsize=(10, 7))
plt.xlabel("Coefficients (%)", fontdict = {'fontsize' : 15})
plt.ylabel("X predictors", fontdict = {'fontsize' : 15})
plt.yticks(fontsize=12)
plt.xticks(fontsize=12)
plt.title(('Coefficients of X predictors from Ridge Regression model\n'), weight = 'bold', fontdict = {'fontsize' : 18});

In [ ]:
# The coefficients of the X variable in our model
ridge_coefs = pd.DataFrame({'X variable':X_train.columns,
                            'coefficients':ridge.coef_,
                            'exp_coef': np.exp(ridge.coef_),
                            'coef (%)':((np.exp(ridge.coef_)-1)*100)})
ridge_coefs.sort_values('coef (%)', inplace=True, ascending=False)
ridge_coefs.head(20)

In [ ]:
# The intercept in our model
ridge.intercept_

**Interpretation of coefficients and intercept**

**Coefficients**

While holding all else constant, it is interpreted as for every 1 unit increase in the x predictor, we expect to observe an increase of the value percentage in y target variable. While for the interaction term, an increase in 1 unit in the x predictor, we expect to observe an increase of the value percentage in y target variable and an increase in the other x variable that it has interaction term with. 

With reference to table above, a unit increase in the above grade living area (gr_liv_area) increases 11.18% in sale price. Thus the top 5 features the sales price is heavily affected by the changes are above grade living area, overall quality of the house, ms zoning at Residential Low Density, the interaction of total basement area and the quality of the basement as well as the fireplace quality. 

The two zones that the sale price is affected mainly at Residential Low Density and Floating Village Residential with each unit change resulting in 6.5% and 3.3% increase in sale price compared to the other zones with 0-1.8% increase. Interestingly, kitchen quality, garage area and heating quality plays a significant role in sale price that results in an increase of 3.2-4.8% in sale price for every unit increase. Meanwhile, there are a few features that has very low correlation such as basement quality and full bath that has very minimum changes to sales prices. This is in particular with full bath that causes a decrease in sale price by 1% for each unit increase despite full bath is one of the top consideration of a buyer.

Therefore based on the model prediction, It is best to have a minimum amount of full bath with a focus on the features of the residential zome of low density or floating village, above grade living area, overall quality of the house, fireplace as well as the basement area and its quality. As these features heavily impacted the change in saleprice of the house.

**Intercept**

When the x predictors are at zero, we expect the log(y) to intercept at 12.011. 


#### 4.5.2 To evaluate how our model fair in prediction and also with null model

In [ ]:
print(f'MSE on train set: {mean_squared_error(np.exp(y_train), np.exp(ridge.predict(z_train)))}')
print(f'MSE on test set: {mean_squared_error(np.exp(y_val),np.exp(ridge.predict(z_val)))}')

**Analysis: The MSE represents the average distance squared from the predicted value. Based on the results above, the MSE for train set and test set are similar, which means that the model is not likely to overfit to the data.**

In [ ]:
print(f'RMSE on train set: {np.sqrt(mean_squared_error(np.exp(y_train), np.exp(ridge.predict(z_train))))}')
print(f'RMSE on test set: {np.sqrt(mean_squared_error(np.exp(y_val),np.exp(ridge.predict(z_val))))}')

**Analysis: Based on the RSME value, it is approximately +- 23000 unit differences from the predicted value for the model**

In [ ]:
print(f'R^2 on train set: {r2_score(np.exp(y_train), np.exp(ridge.predict(z_train)))}')
print(f'R^2 on test set: {r2_score(np.exp(y_val), np.exp(ridge.predict(z_val)))}')

**Analysis: Based on the R2 value, 89% of the observed variation can be explained by the model's inputs.**

In [ ]:
print(f'MSE of baseline model: {mean_squared_error(np.exp(y_val), [np.mean(np.exp(y_train))] * len(y_val))}')

In [ ]:
print(f'R^2 of baseline model: {r2_score(np.exp(y_val), [np.mean(np.exp(y_train))] * len(y_val))}')

**Analysis: With reference to the MSE and R^2 value of the baseline model, our model is a fairly good model with an R^2 value with 0.89 and a much smaller MSE value compared with the baseline model with R^2 value of 0 and a very big MSE value.** 

#### 4.5.3 To check on the LINE Assumptions

- L - Linear relationship
- I - Independent errors
- N - Normally distributed errors
- E - Equal variance of errors (homoscedasticity)
- M - No Multicollinearity/Independence of Predictors

In [ ]:
# For linear relationship
plt.figure(figsize = (10,5))
plt.scatter(train_housing_data["gr_liv_area"], y_data, s = 1)
plt.xlabel("above grade living area (sqft)", fontdict = {'fontsize' : 12})
plt.ylabel("log_sale price", fontdict = {'fontsize' : 12})
plt.title(('Distribution of above grade living area with log_scale price\n'), weight = 'bold', fontdict = {'fontsize' : 15});

**Analysis: The scatterplot indicates that the  samples are approximately normally distributed.**

In [ ]:
# make y predictions
y_pred = ridge.predict(z_val)

**For independence**
Yes, the sample are assumed to be indepedent.

In [ ]:
# Normally distributed
residuals = np.exp(y_val) - np.exp(y_pred)
plt.hist(residuals, bins=50);
plt.xlabel("log_scale price", fontdict = {'fontsize' : 12})
plt.ylabel("log_sale price", fontdict = {'fontsize' : 12})
plt.title(('Distribution of above residuals of log_sale price\n'), weight = 'bold', fontdict = {'fontsize' : 15});

**Analysis: The histogram indicates that the samples are normally distributed.**

In [ ]:
# Evenly variance of error
plt.scatter(np.exp(y_pred), residuals, s = 3)
plt.axhline(0, color = 'orange')
plt.xlabel("log_sale price", fontdict = {'fontsize' : 12})
plt.ylabel("residuals", fontdict = {'fontsize' : 12})
plt.title(('Distribution of residuals of log_sale price\n'), weight = 'bold', fontdict = {'fontsize' : 15});

**Analysis: The residuals are approximately evenly distributed**

In [ ]:
# Multicolinearity
plt.figure(figsize = [40,50])
res = sns.heatmap(train_housing_data.corr(), annot = True, mask = np.triu(train_housing_data.corr()), vmin = -1, vmax =1, cmap = 'coolwarm', fmt='.1g')
sns.set(font_scale=1.8)

**Analysis: There are no strong correlation between x-predictors unless within the nominal categorical data or there is an interaction term associated and developed using that x-predictors**

## Conclusions 

In conclusion, we have addressed our problem statement stated in the beginning, we have first performed data cleaning for the datasets followed by exploring and visualizing the data using exploratory data analysis. We then performed extensive feature engineering on various attributes that have resulted in an increase correlation to the house prices, such as total basement area and basement quality, garage area and garage finish, fireplace quality and heating quality. 

Based on the engineered data, we built three linear regression models: Linear Regression, Lasso and Ridge. The three models yield similar and stable results for the cross validation score. We have chosen Ridge as our model as the cross validation score for this model is the highest out of the three, and this model is useful for our analysis in which we have many predictors that may have a high degree of Multicollinearity between each other. The model accomplished an R2 score of approximately 89% on training sets and 90% on training-validation sets with an RMSE of 23395 and 22741 respectively. 

Using this model, we are able to identify the key features that are affecting the changes of the sale price of the house and utilise it to structure key features into future affordable and marketable homes.

## Recommendations

Overall, the top 5 features of the property that will affect the changes of the sale price are above grade living area, overall quality of the house, ms zoning at Residential Low Density, the interaction of total basement area and the quality of the basement as well as the fireplace quality. Interestingly, kitchen quality, garage area and heating quality were also identified as key features that enhance the sale price of the house. Meanwhile, it is best to have a minimum amount of full bath required in the house as it causes a decrease in sale price by 1% for each unit increase although full bath is listed as one of the buyer's feature consideration.

With these insights, our construction company in Ames Iowa is able to utilise this domain knowledge to build a affordable and desirable houses and advise homeowners to renovate their house focusing on key features that will increase their sale price, as well as estimate and predict the sale price given the features.

## References

[1] "From Linear Regression to Ridge Regression, the Lasso, and the Elastic Net,"2020. [Online]. Available: https://towardsdatascience.com/from-linear-regression-to-ridge-regression-the-lasso-and-the-elastic-net-4eaecaf5f7e6 [Accessed: Apr. 8. 2021].